In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained TensorFlow model
#model = load_model('emotion_detection_model.h5')
#model = load_model('efficientnet_model.h5')
model = load_model('densenet_model.h5')

# Define the preprocessing function for TensorFlow (same as during training)
def preprocess_image(image):
    #img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert BGR to RGB For Efficient Net + Custom
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (3 channels) For Dense Net
    img = cv2.resize(img, (48, 48))  # Resize to model's expected input size
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    img = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
    return img

# Start webcam capture
video_capture = cv2.VideoCapture(0)

# Emotion labels (adjust to match the labels your model uses)
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", 'Neutral','Sad','Surprise']

while True:
    ret, frame = video_capture.read()  # Capture a frame from the webcam
    
    # Preprocess the frame for the model
    img = preprocess_image(frame)

    # Runs model to get predictions
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Display the predicted emotion on the frame
    predicted_emotion = emotion_labels[predicted_class]
    cv2.putText(frame, f'Emotion: {predicted_emotion}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame with emotion label
    cv2.imshow('Emotion Recognition', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━

KeyboardInterrupt: 

In [1]:
import cv2
import numpy as np
import onnxruntime as ort

# Load the ONNX model
onnx_model_path = 'dense_model.onnx'  # Path to your ONNX model
session = ort.InferenceSession(onnx_model_path)

# Define the preprocessing function for ONNX (same as during training)
def preprocess_image(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (3 channels) For Dense Net
    img = cv2.resize(img, (48, 48))  # Resize to model's expected input size
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = np.array(img, dtype=np.float32)  # Ensure the data type is float32
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return img

# Start webcam capture
video_capture = cv2.VideoCapture(0)

# Emotion labels (adjust to match the labels your model uses)
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", 'Neutral', 'Sad', 'Surprise']

while True:
    ret, frame = video_capture.read()  # Capture a frame from the webcam
    
    # Preprocess the frame for the model
    img = preprocess_image(frame)

    # Prepare the input for the model (ONNX expects a dictionary of input names)
    inputs = {session.get_inputs()[0].name: img}

    # Run the model to get predictions
    predictions = session.run(None, inputs)  # None means we want all outputs
    predicted_class = np.argmax(predictions[0], axis=1)[0]

    # Display the predicted emotion on the frame
    predicted_emotion = emotion_labels[predicted_class]
    cv2.putText(frame, f'Emotion: {predicted_emotion}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame with emotion label
    cv2.imshow('Emotion Recognition', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 